# Naive Bayes

***

## Índice

1. [O teorema de Bayes](#O-teorema-de-Bayes)
2. [Importando bibliotecas](#Importando-bibliotecas)
3. [Distribuição dos dados](#Distribuição-dos-dados)
4. [Distribuição de Bernoulli](#Distribuição-de-Bernoulli)

## O teorema de Bayes

O teorema de Bayes define que: a probabilidade de um evento $y$ ocorrer a partir de uma condição $X$, é a probabilidade da condição $X$ ocorrer dado que o evento $y$ tenha ocorrido, multiplicidao pela probabilidade do evento $y$ ocorrer, dividido pela probabilidade da condição $X$ ocorrer. Representado pela seguinte formula

$$
    P(y|X) = \frac{P(X|y) \times P(y)}{P(X)}, X = \{x_1, x_2 ... x_n\} \to P(y | x_1, x_2 ... x_n) = \frac{P(y) \times \prod_{i=1}^nP(x_i|y)}{\prod_{i=1}^nP(x_i)}
$$

Mas como isso se aplica em nossos problemas de classificação?

Para escolher uma classe $y$, é utilizada a seguinte regra:

$$
    \hat{y} = argmax_y\{P(y) \times \prod_{i=1}^nP(x_i|y)\}
$$

Essa fórmula é semelhante ao de teorema de Bayes, a diferença é que como $\prod_{i=1}^nP(x_i)$ é um termo constante para todas as classes, podemos remover da equação de decisão.

Vamos concretizar o evento $y$ como sendo a ocorrência de uma classe $c_1$ e associar a condição $X$ ao teor alcoólico do vinho, variável $alcohol$, e à intensidade de cor, $color\_intensity$:

$$
    P(y=c_1|alcohol, color\_intensity) = \frac{P(alcohol, color\_intensity|y=c_1) \times P(y=c_1)}{P(alcohol) \times P(color\_intensity)} = \frac{P(alcohol|y=c_1) \times P(color\_intensity|y=c_1) \times P(y=c_1)}{P(alcohol) \times P(color\_intensity)}
$$


## Importando bibliotecas

In [ ]:
# Bibliotecas de manipualção e visualização de dados
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

# Classes dos modelo
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Funções de avaliação dos modelos
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# Função para carregar nosso dataset
from sklearn.datasets import load_wine

# Função para criar distribuição de Bernoulli
from scipy.stats import bernoulli

def show_decision_region(x, y, clf, f0, f1):
    plot_decision_regions(x, y, clf=clf)
    plt.xlabel(f0)
    plt.ylabel(f1)
    if clf.__class__.__name__ == "KNeighborsClassifier":
        plt.title(clf.__class__.__name__ + " k = " + str(clf.n_neighbors))
    else:
        plt.title(clf.__class__.__name__)
    plt.show()

In [ ]:
# carregar dataset
X, y = load_wine(return_X_y=True, as_frame=True)

# definição de classes e features
class_a = 0
class_b = 1
feature_0 = "alcohol"
feature_1 = "color_intensity"

# filtrar classes e features
class_0_instances = (y == class_a)
class_1_instances = (y == class_b)

filtered_y = y[class_0_instances | class_1_instances]
filtered_X = X[class_0_instances | class_1_instances]
filtered_X = filtered_X[[feature_0, feature_1]]

# dividir classificador em treino e teste
X_train, X_test, y_train, y_test = train_test_split(filtered_X[[feature_0, feature_1]], filtered_y, test_size=0.3, random_state=199)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=199)

In [ ]:
colors = {0: "steelblue", 1: "darkorange"}
markers = {0: "s", 1: "^"}

class_0_instances = (y == 0)
class_1_instances = (y == 1)
feature_0 = "alcohol"
feature_1 = "color_intensity"

filtered_y = y[class_0_instances | class_1_instances]
filtered_X = X[class_0_instances | class_1_instances]
filtered_X = filtered_X[[feature_0, feature_1]]

plt.scatter(X[feature_0][class_0_instances], X[feature_1][class_0_instances], c=colors[0], marker=markers[0])
plt.scatter(X[feature_0][class_1_instances], X[feature_1][class_1_instances], c=colors[1], marker=markers[1])

plt.xlabel("alcohol")
plt.ylabel("color_intensity")

## Distribuição dos dados

Saber como as nossas variáveis se comportam é essencial para o classificador bayesiano, já que o classificador utiliza a distribuição de probabilidade $P(X|y)$ para fazer a classificação, precisamos entender o comportamento da distribuição $P(alcohol, color\_intensity|y)$ para saber qual família de classificadores bayesianos iremos aplicar.

Cada classificador utiliza uma suposição diferente da função $P(X|y)$:

- Features contínuas: $GuassianNB$
- Distribuições com features discretas, ou uma contagem (0, 1, 2, 3, ...): $MultinomialNB$ ou $ComplementNB$
- Para features categóricas: $CategoricalNB$
- Para features como distribuições de Bernoulli: $BernoulliNB$

Essas diferentes suposições resultam em [diferentes regras de decisão](https://scikit-learn.org/stable/modules/naive_bayes.html#bernoulli-naive-bayes).


In [ ]:
# qual a ditribuição da variável alcohol para as duas classes?

plot_data = filtered_X
plot_data["class"] = filtered_y

# visualizando a distribuição da variável alcohol para as duas classes
sns.displot(plot_data, x=feature_0, hue="class", kind="kde")
plt.show()
# calculando a média e desvio padrão da feature alcohol para as duas classes
print("feature_0 class_0 mean: %.4f, std: %.4f" %(filtered_X[feature_0][class_0_instances].mean(), filtered_X[feature_0][class_0_instances].std()))
print("feature_0 class_1 mean: %.4f, std: %.4f" %(filtered_X[feature_0][class_1_instances].mean(), filtered_X[feature_0][class_1_instances].std()))

# visualizando a distribuição da variável color_intensity para as duas classes
sns.displot(plot_data, x=feature_1, hue="class", kind="kde")
plt.show()
# calculando a média e desvio padrão da feature color_intensity para as duas classes
print("feature_1 class_0 mean: %.4f, std: %.4f" %(filtered_X[feature_1][class_0_instances].mean(), filtered_X[feature_1][class_0_instances].std()))
print("feature_1 class_1 mean: %.4f, std: %.4f" %(filtered_X[feature_1][class_1_instances].mean(), filtered_X[feature_1][class_1_instances].std()))

No nosso caso, as distribuições se assemelham a uma distribuição normal. Portanto vamos utilizar o $GuassianNaiveBayes$.

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)

print(classification_report(model.predict(X_test), y_test))

show_decision_region(
    np.array(
        [
            X_test[feature_0].values, 
            X_test[feature_1].values,
        ]
    ).T, 
    y_test.values, 
    model, 
    feature_0, 
    feature_1
)

## Distribuição de Bernoulli

Vamos avaliar outro tipo de distribuição, a distribuição de Bernoulli. A distribuição de Bernoulli é mais simples de visualizarmos a regra de decisão.

A distribuição de Bernoulli segue a equação abaixo, que atribui uma probabilidade de $1-p$ do evento $X=0$ e $p$ para $X=1$.

$$
    P(X=k) =
    \begin{cases}
        1-p   & \quad k = 0\\
        p   & \quad k = 1
    \end{cases}
$$

A regra de decisão para um classificador de Bayes com a distribuição de Bernoulli segue a equação:

$$
    P(x_i | y) = P(i | y) \times x_i + (1-P(i | y))\times(1-x_i)
$$

In [ ]:
# criando duas features com a distribuição de Bernoulli
np.random.seed(199)
x1 = bernoulli.rvs(0.2, size=100)
x2 = bernoulli.rvs(0.4, size=100)

# criando um dataset com as duas distribuições
X = np.stack([x1, x2], axis=1)

# criando classes para essa distribuição
y = bernoulli.rvs(0.3, size=100)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=199)

# visualizando a distribuição de x1
_, bins, _ = plt.hist(
    [x1[y==0], x1[y==1]],
    2,
    alpha=0.5,
    histtype="bar",
    ec="black",
    label=["Class 0", "Class 1"],
    align="left",
    color=["g", "r"],
    stacked=True,
)
plt.legend(loc="upper right")
plt.show()

In [ ]:
# treinando o modelo
model = BernoulliNB()
model.fit(X_train, y_train)

print(classification_report(model.predict(X_test), y_test))